In [2]:
## This file uses all generated data, so it needs to be run after the files that use kaggle data

In [1]:
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
coach_features = pd.read_csv('data/2023/generated_data/coach_features.csv')
stationary_probs_all = pd.read_csv('data/2023/generated_data/stationary_probabilities.csv')
stationary_probs = stationary_probs_all[stationary_probs_all['Season'] >= 2003]
team_season_features = pd.read_csv('data/2023/generated_data/team_season_features.csv')
additional_team_features = pd.read_csv('data/2023/generated_data/additional_team_features.csv')
preseason_rankings = pd.read_csv('data/2023/generated_data/preseason_rankings.csv')

In [3]:
print(coach_features.shape)
coach_features.head()

(7254, 12)


,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,rk
0,1101,2014,abilene-christian,0,0,0,0.355,0,0,0,0,1
1,1101,2015,abilene-christian,0,0,0,0.339,0,0,0,0,1
2,1101,2016,abilene-christian,0,0,0,0.366,0,0,0,0,1
3,1101,2017,abilene-christian,0,0,0,0.385,0,0,0,0,1
4,1101,2018,abilene-christian,0,0,0,0.409,0,0,0,0,1


In [4]:
print(stationary_probs.shape)
stationary_probs.head()

(7255, 4)


,Unnamed: 0,Season,TeamID,pi_i
5407,5407,2003,1102,0.002818
5408,5408,2003,1103,0.002689
5409,5409,2003,1104,0.004730
5410,5410,2003,1105,0.001254
5411,5411,2003,1106,0.001915


In [5]:
print(team_season_features.shape)
team_season_features.head()

(7247, 38)


,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,...,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement
0,2003,1102,28,12,16,0.428571,57.250000,57.000000,0.250000,15.878163,...,0.0,7.0,9.0,8.0,158.0,149.0,-9.0,160.0,151.0,-9.0
1,2003,1103,27,13,14,0.481481,78.777778,78.148148,0.629630,11.126039,...,1.0,1.0,13.0,2.0,182.0,198.0,16.0,163.0,187.0,24.0
2,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,...,1.0,6.0,5.0,11.0,38.0,27.0,-11.0,33.0,39.0,6.0
3,2003,1105,26,7,19,0.269231,71.769231,76.653846,-4.884615,15.552212,...,0.0,3.0,16.0,4.0,313.0,316.0,3.0,307.0,313.0,6.0
4,2003,1106,28,13,15,0.464286,63.607143,63.750000,-0.142857,12.374266,...,0.0,4.0,11.0,5.0,248.0,236.0,-12.0,263.0,227.0,-36.0


In [6]:
print(additional_team_features.shape)
additional_team_features.head()

(1344, 5)


,TeamID,Season,school,returning_min_pct,returning_score_pct
0,1103,2022,akron,60.1,53.1
1,1104,2022,alabama,46.6,49.3
2,1412,2022,alabama-birmingham,60.0,63.2
3,1112,2022,arizona,48.3,45.9
4,1116,2022,arkansas,37.5,37.4


In [6]:
additional_team_features.drop_duplicates(inplace=True)

In [7]:
idx = ['Season', 'TeamID']
features_all = team_season_features.merge(stationary_probs, on=idx)\
.merge(coach_features, on=idx)\
.merge(additional_team_features, on=idx)\
.merge(preseason_rankings, on=idx, how='left')
features_all['preseason_pts'] = features_all['preseason_pts'].fillna(0)
features_all.head()

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,...,tourneys_car,sw16_car,ff_car,champ_car,rk,school_y,returning_min_pct,returning_score_pct,school,preseason_pts
0,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,...,4,0,0,0,1,alabama,80.5,78.0,alabama,1295.0
1,2003,1112,28,25,3,0.892857,85.214286,70.250000,14.964286,12.330225,...,24,12,5,1,1,arizona,98.4,99.0,arizona,1773.0
2,2003,1113,29,18,11,0.620690,75.965517,69.172414,6.793103,16.889865,...,3,0,0,0,1,arizona-state,73.2,66.6,arizona state,4.0
3,2003,1120,30,19,11,0.633333,70.100000,65.566667,4.533333,18.358891,...,8,3,0,0,1,auburn,67.8,70.9,NaN,0.0
4,2003,1122,27,20,7,0.740741,69.703704,67.703704,2.000000,12.584529,...,2,0,0,0,1,austin-peay,73.5,62.7,NaN,0.0


In [8]:
print(features_all.shape)

(1336, 55)


In [9]:
features_all[features_all['Season']==2023]

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,...,tourneys_car,sw16_car,ff_car,champ_car,rk,school_y,returning_min_pct,returning_score_pct,school,preseason_pts
1268,2023,1104,34,29,5,0.852941,82.176471,68.500000,13.676471,17.443557,...,6,1,0,0,1,alabama,40.7,41.0,alabama,281.0
1269,2023,1112,34,28,6,0.823529,82.676471,71.500000,11.176471,14.178423,...,2,1,0,0,1,arizona,45.3,44.6,arizona,543.0
1270,2023,1113,34,22,12,0.647059,70.323529,67.705882,2.617647,13.847945,...,4,0,0,0,1,arizona-state,48.8,40.4,NaN,0.0
1271,2023,1116,33,20,13,0.606061,74.424242,67.393939,7.030303,14.819080,...,6,3,0,0,1,arkansas,16.6,13.0,arkansas,1026.0
1272,2023,1120,32,20,12,0.625000,72.718750,67.062500,5.656250,13.823914,...,12,5,1,0,1,auburn,63.2,56.6,auburn,623.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,2023,1433,34,27,7,0.794118,71.441176,62.911765,8.529412,10.968498,...,3,0,0,0,1,virginia-commonwealth,41.4,36.1,NaN,0.0
1332,2023,1436,32,22,10,0.687500,72.156250,66.718750,5.437500,15.668315,...,5,0,0,0,1,vermont,40.2,33.5,NaN,0.0
1333,2023,1438,32,25,7,0.781250,67.812500,60.250000,7.562500,11.717875,...,11,4,1,1,1,virginia,87.8,91.4,virginia,462.0
1334,2023,1452,33,19,14,0.575758,76.303030,71.030303,5.272727,15.531537,...,26,9,2,0,1,west-virginia,17.1,11.5,NaN,0.0


In [10]:
features_all[features_all['Season'] == 2023].sort_values(by='pi_i', ascending=False)[['Season','school_y','pi_i']].head(50)

,Season,school_y,pi_i
1268,2023,alabama,0.007706
1286,2023,houston,0.007344
1326,2023,ucla,0.007121
1277,2023,connecticut,0.006619
1323,2023,texas,0.006538
1315,2023,purdue,0.006366
1322,2023,tennessee,0.006319
1269,2023,arizona,0.006254
1293,2023,kansas,0.006052
1284,2023,gonzaga,0.005934


In [11]:
# removed avg height and avg year this year
features = features_all[['Season', 'TeamID', 'Seed', 'total_games', 'win_pct', 'avg_points_for', 
              'avg_points_against', 'avg_game_margin', 'std_game_margin', 
              'non_home_wins', 'home_losses', 'close_wins', 'close_losses', 
              '3FGA_pg', '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 
              'off_eff', 'def_eff', 'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
              'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
              'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
              'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
              'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
              'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
              'returning_min_pct', 'returning_score_pct', 'preseason_pts']]

In [12]:
print(features.shape)
features.head()

(1336, 44)


,Season,TeamID,Seed,total_games,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,...,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,returning_min_pct,returning_score_pct,preseason_pts
0,2003,1104,Y10,28,0.607143,69.285714,65.000000,4.285714,13.149843,4.0,...,1,0,0.660,4,0,0,0,80.5,78.0,1295.0
1,2003,1112,Z01,28,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,...,0,0,0.738,24,12,5,1,98.4,99.0,1773.0
2,2003,1113,Z10,29,0.620690,75.965517,69.172414,6.793103,16.889865,6.0,...,0,0,0.520,3,0,0,0,73.2,66.6,4.0
3,2003,1120,W10,30,0.633333,70.100000,65.566667,4.533333,18.358891,6.0,...,0,0,0.617,8,3,0,0,67.8,70.9,0.0
4,2003,1122,W13,27,0.740741,69.703704,67.703704,2.000000,12.584529,9.0,...,0,0,0.522,2,0,0,0,73.5,62.7,0.0


## Tourney games

In [13]:
tourney_games_all = pd.read_csv('data/2023/kaggle_data/MNCAATourneyCompactResults.csv')
tourney_games = tourney_games_all[tourney_games_all['Season'] >= 2003]
tourney_games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1136,2003,134,1421,92,1411,84,N,1
1137,2003,136,1112,80,1436,51,N,0
1138,2003,136,1113,84,1272,71,N,0
1139,2003,136,1141,79,1166,73,N,0
1140,2003,136,1143,76,1301,74,N,1


In [14]:
tourney_games['rand'] = [random.random() for i in range(len(tourney_games))]
tourney_games['teams'] = [(t1, t2) if r >= 0.5 else (t2, t1) for t1, t2, r in \
                          zip(tourney_games['WTeamID'], tourney_games['LTeamID'], tourney_games['rand'])]
tourney_games['t1_score_diff'] = [t1-t2 if r >= 0.5 else t2-t1 for t1, t2, r in \
                          zip(tourney_games['WScore'], tourney_games['LScore'], tourney_games['rand'])]
tourney_games[['team1','team2']] = pd.DataFrame(tourney_games['teams'].tolist(), index=tourney_games.index)

In [15]:
tourney_games

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,rand,teams,t1_score_diff,team1,team2
1136,2003,134,1421,92,1411,84,N,1,0.425811,"(1411, 1421)",-8,1411,1421
1137,2003,136,1112,80,1436,51,N,0,0.377668,"(1436, 1112)",-29,1436,1112
1138,2003,136,1113,84,1272,71,N,0,0.982432,"(1113, 1272)",13,1113,1272
1139,2003,136,1141,79,1166,73,N,0,0.636381,"(1141, 1166)",6,1141,1166
1140,2003,136,1143,76,1301,74,N,1,0.174111,"(1301, 1143)",-2,1301,1143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2379,2022,146,1242,76,1274,50,N,0,0.522222,"(1242, 1274)",26,1242,1274
2380,2022,146,1314,69,1389,49,N,0,0.227636,"(1389, 1314)",-20,1389,1314
2381,2022,152,1242,81,1437,65,N,0,0.673131,"(1242, 1437)",16,1242,1437
2382,2022,152,1314,81,1181,77,N,0,0.502203,"(1314, 1181)",4,1314,1181


In [16]:
tourney_results = tourney_games[['Season', 'team1', 'team2', 't1_score_diff']]
tourney_results['is_t1_win'] = [diff > 0 for diff in tourney_results['t1_score_diff']]

In [17]:
tourney_results

,Season,team1,team2,t1_score_diff,is_t1_win
1136,2003,1411,1421,-8,False
1137,2003,1436,1112,-29,False
1138,2003,1113,1272,13,True
1139,2003,1141,1166,6,True
1140,2003,1301,1143,-2,False
...,...,...,...,...,...
2379,2022,1242,1274,26,True
2380,2022,1389,1314,-20,False
2381,2022,1242,1437,16,True
2382,2022,1314,1181,4,True


In [18]:
matchup_features_all = tourney_results.merge(features, left_on=['Season', 'team1'], right_on=['Season', 'TeamID'])\
.merge(features, left_on=['Season', 'team2'], right_on=['Season', 'TeamID'], suffixes=['_1', '_2'])
print(matchup_features_all.shape)
matchup_features_all.head()

(1248, 91)


,Season,team1,team2,t1_score_diff,is_t1_win,TeamID_1,Seed_1,total_games_1,win_pct_1,avg_points_for_1,...,is_ap_pre_top_15_2,is_ap_pre_top_25_2,coach_WL_car_2,tourneys_car_2,sw16_car_2,ff_car_2,champ_car_2,returning_min_pct_2,returning_score_pct_2,preseason_pts_2
0,2003,1411,1421,-8,False,1411,X16a,30,0.600000,72.800000,...,0,0,0.463,1,0,0,0,58.1,59.9,0.0
1,2003,1400,1421,21,True,1400,X01,28,0.785714,78.857143,...,0,0,0.463,1,0,0,0,58.1,59.9,0.0
2,2003,1436,1112,-29,False,1436,Z16,29,0.655172,67.793103,...,0,0,0.738,24,12,5,1,98.4,99.0,1773.0
3,2003,1211,1112,-1,False,1211,Z09,31,0.741935,77.064516,...,0,0,0.738,24,12,5,1,98.4,99.0,1773.0
4,2003,1113,1272,13,True,1113,Z10,29,0.620690,75.965517,...,0,0,0.721,6,3,1,0,NaN,NaN,2.0


In [20]:
features.columns

Index(['Season', 'TeamID', 'Seed', 'total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'preseason_pts'],
      dtype='object')

In [21]:
difference_cols = ['total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'preseason_pts']

In [22]:
for col in difference_cols:
    matchup_features_all['diff_'+col] = matchup_features_all[col+'_1'] - matchup_features_all[col+'_2']

In [23]:
pd.set_option("display.max_columns", None)
matchup_features_all.head()

,Season,team1,team2,t1_score_diff,is_t1_win,TeamID_1,Seed_1,total_games_1,win_pct_1,avg_points_for_1,avg_points_against_1,avg_game_margin_1,std_game_margin_1,non_home_wins_1,home_losses_1,close_wins_1,close_losses_1,3FGA_pg_1,3_pct_1,FTA_pg_1,FT_pct_1,TOV_pg_1,avg_pace_1,off_eff_1,def_eff_1,pythag_win_pct_1,luck_1,wins_vs_tourney_teams_1,away_wins_vs_tourney_teams_1,losses_vs_tourney_teams_1,losses_vs_non_tourney_teams_1,games_vs_tourney_teams_1,final_net_1,net_improvement_1,final_pom_1,pom_improvement_1,pi_i_1,is_ap_pre_top_5_1,is_ap_pre_top_15_1,is_ap_pre_top_25_1,coach_WL_car_1,tourneys_car_1,sw16_car_1,ff_car_1,champ_car_1,returning_min_pct_1,returning_score_pct_1,preseason_pts_1,TeamID_2,Seed_2,total_games_2,win_pct_2,avg_points_for_2,avg_points_against_2,avg_game_margin_2,std_game_margin_2,non_home_wins_2,home_losses_2,close_wins_2,close_losses_2,3FGA_pg_2,3_pct_2,FTA_pg_2,FT_pct_2,TOV_pg_2,avg_pace_2,off_eff_2,def_eff_2,pythag_win_pct_2,luck_2,wins_vs_tourney_teams_2,away_wins_vs_tourney_teams_2,losses_vs_tourney_teams_2,losses_vs_non_tourney_teams_2,games_vs_tourney_teams_2,final_net_2,net_improvement_2,final_pom_2,pom_improvement_2,pi_i_2,is_ap_pre_top_5_2,is_ap_pre_top_15_2,is_ap_pre_top_25_2,coach_WL_car_2,tourneys_car_2,sw16_car_2,ff_car_2,champ_car_2,returning_min_pct_2,returning_score_pct_2,preseason_pts_2,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_preseason_pts
0,2003,1411,1421,-8,False,1411,X16a,30,0.600000,72.800000,70.833333,1.966667,11.998565,7.0,3.0,5.0,2.0,18.500000,0.320721,28.066667,0.619952,15.233333,67.262791,108.724747,104.193929,0.594100,0.005900,0.0,0.0,1.0,11.0,1.0,217.0,-14.0,268.0,-15.0,0.002053,0,0,0,0.492,1,0,0,0,69.3,66.9,0.0,1421,X16b,29,0.448276,71.206897,78.448276,-7.241379,17.203959,3.0,2.0,6.0,3.0,18.000000,0.360153,20.931034,0.762768,16.206897,65.815172,106.461081,116.825884,0.206332,0.241944,0.0,0.0,6.0,10.0,6.0,186.0,-3.0,273.0,-7.0,0.001662,0,0,0,0.463,1,0,0,0,58.1,59.9,0.0,1,0.151724,1.593103,-7.614943,9.208046,-5.205395,4.0,1.0,-1.0,-1.0,0.500000,-0.039433,7.135632,-0.142815,-0.973563,1.447619,2.263666,-12.631956,0.387769,-0.236045,0.0,0.0,-5.0,1.0,-5.0,31.0,-11.0,-5.0,-8.0,0.000391,0.029,0,0,0,0,11.2,7.0,0.0
1,2003,1400,1421,21,True,1400,X01,28,0.785714,78.857143,68.678571,10.178571,12.159810,8.0,0.0,2.0,2.0,16.785714,0.348936,23.785714,0.714715,13.428571,66.885714,117.025288,102.741637,0.872403,-0.086688,4.0,1.0,5.0,1.0,9.0,4.0,-3.0,7.0,-2.0,0.006598,1,0,0,0.633,11,3,1,0,90.1,89.0,1422.0,1421,X16b,29,0.448276,71.206897,78.448276,-7.241379,17.203959,3.0,2.0,6.0,3.0,18.000000,0.360153,20.931034,0.762768,16.206897,65.815172,106.461081,116.825884,0.206332,0.241944,0.0,0.0,6.0,10.0,6.0,186.0,-3.0,273.0,-7.0,0.001662,0,0,0,0.463,1,0,0,0,58.1,59.9,0.0,-1,0.337438,7.650246,-9.769704,17.419951,-5.044150,5.0,-2.0,-4.0,-1.0,-1.214286,-0.011217,2.854680,-0.048053,-2.778325,1.070542,10.564206,-14.084247,0.666071,-0.328633,4.0,1.0,-1.0,-9.0,3.0,-182.0,0.0,-266.0,5.0,0.004936,0.170,10,3,1,0,32.0,29.1,1422.0
2,2003,1436,1112,-29,False,1436,Z16,29,0.655172,67.793103,63.137931,4.655172,12.382624,10.0,1.0,3.0,4.0,15.482759,0.340757,19.551724,0.657848,14.068966,62.374179,108.035787,100.640934,0.728997,-0.073825,0.0,0.0,1.0,9.0,1.0,144.0,41.0,144.0,38.0,0.003165,0,0,0,0.453,1,0,0,0,61.2,52.9,0.0,1112,Z01,28,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,1.0,3.0,2.0,20.0714

In [24]:
diff_cols = [col for col in matchup_features_all.columns if 'diff' in col]
matchup_cols = ['Season', 'TeamID_1', 'TeamID_2', 'Seed_1', 'Seed_2'] + diff_cols

In [28]:
diff_cols

['t1_score_diff',
 'diff_total_games',
 'diff_win_pct',
 'diff_avg_points_for',
 'diff_avg_points_against',
 'diff_avg_game_margin',
 'diff_std_game_margin',
 'diff_non_home_wins',
 'diff_home_losses',
 'diff_close_wins',
 'diff_close_losses',
 'diff_3FGA_pg',
 'diff_3_pct',
 'diff_FTA_pg',
 'diff_FT_pct',
 'diff_TOV_pg',
 'diff_avg_pace',
 'diff_off_eff',
 'diff_def_eff',
 'diff_pythag_win_pct',
 'diff_luck',
 'diff_wins_vs_tourney_teams',
 'diff_away_wins_vs_tourney_teams',
 'diff_losses_vs_tourney_teams',
 'diff_losses_vs_non_tourney_teams',
 'diff_games_vs_tourney_teams',
 'diff_final_net',
 'diff_net_improvement',
 'diff_final_pom',
 'diff_pom_improvement',
 'diff_pi_i',
 'diff_coach_WL_car',
 'diff_tourneys_car',
 'diff_sw16_car',
 'diff_ff_car',
 'diff_champ_car',
 'diff_returning_min_pct',
 'diff_returning_score_pct',
 'diff_preseason_pts']

In [33]:
feature_metadata = [
  {
    'col': x,
    'name': x.replace('diff_','').replace('_',' ').title(),
    'description': ''
   } for x in diff_cols if x != 't1_score_diff'
]

In [34]:
feature_metadata

[{'col': 'diff_total_games', 'name': 'Total Games', 'description': ''},
 {'col': 'diff_win_pct', 'name': 'Win Pct', 'description': ''},
 {'col': 'diff_avg_points_for', 'name': 'Avg Points For', 'description': ''},
 {'col': 'diff_avg_points_against',
  'name': 'Avg Points Against',
  'description': ''},
 {'col': 'diff_avg_game_margin', 'name': 'Avg Game Margin', 'description': ''},
 {'col': 'diff_std_game_margin', 'name': 'Std Game Margin', 'description': ''},
 {'col': 'diff_non_home_wins', 'name': 'Non Home Wins', 'description': ''},
 {'col': 'diff_home_losses', 'name': 'Home Losses', 'description': ''},
 {'col': 'diff_close_wins', 'name': 'Close Wins', 'description': ''},
 {'col': 'diff_close_losses', 'name': 'Close Losses', 'description': ''},
 {'col': 'diff_3FGA_pg', 'name': '3Fga Pg', 'description': ''},
 {'col': 'diff_3_pct', 'name': '3 Pct', 'description': ''},
 {'col': 'diff_FTA_pg', 'name': 'Fta Pg', 'description': ''},
 {'col': 'diff_FT_pct', 'name': 'Ft Pct', 'description': '

In [35]:
import json
with open('data/2023/model_data/feature_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(feature_metadata, f, ensure_ascii=False, indent=4)

In [25]:
matchup_features_basic = matchup_features_all[matchup_cols]

In [26]:
print(matchup_features_basic.shape)

(1248, 44)


In [137]:
matchup_features_basic.to_csv('data/2023/model_data/matchup_features.csv', index=False)

In [136]:
matchup_features_basic.head()

,Season,TeamID_1,TeamID_2,Seed_1,Seed_2,t1_score_diff,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_preseason_pts
0,2003,1411,1421,X16a,X16b,-8,1,0.151724,1.593103,-7.614943,9.208046,-5.205395,4.0,1.0,-1.0,-1.0,0.500000,-0.039433,7.135632,-0.142815,-0.973563,1.447619,2.263666,-12.631956,0.387769,-0.236045,0.0,0.0,-5.0,1.0,-5.0,31.0,-11.0,-5.0,-8.0,0.000391,0.029,0,0,0,0,11.2,7.0,0.0
1,2003,1436,1112,Z16,Z01,-29,1,-0.237685,-17.421182,-7.112069,-10.309113,0.052399,-1.0,0.0,0.0,2.0,-4.588670,-0.009777,-5.448276,-0.043580,-0.716749,-10.195649,-8.539513,4.232186,-0.207207,-0.030478,-10.0,-5.0,-1.0,8.0,-11.0,142.0,35.0,141.0,38.0,-0.004756,-0.285,-23,-12,-5,-1,-37.2,-46.1,-1773.0
2,2003,1323,1112,Z05,Z01,-17,3,-0.183180,-5.117512,0.943548,-6.061060,2.866029,-3.0,1.0,2.0,0.0,1.702765,0.030207,-2.161290,0.058458,-2.011521,-4.466768,-0.959411,7.112776,-0.098797,-0.084383,-5.0,-3.0,4.0,2.0,-1.0,12.0,-16.0,19.0,-10.0,-0.001456,-0.073,-19,-11,-5,-1,-43.2,-49.7,-1715.0
3,2003,1113,1272,Z10,Z07,13,0,-0.172414,1.448276,3.344828,-1.896552,6.116003,-3.0,1.0,1.0,-1.0,-7.482759,-0.030989,3.310345,0.016122,0.206897,-1.004653,4.182093,5.603289,-0.062388,-0.110026,0.0,0.0,7.0,-2.0,7.0,6.0,-16.0,12.0,-37.0,-0.000375,-0.201,-3,-3,-1,0,NaN,NaN,2.0
4,2003,1113,1242,Z10,Z02,-32,-1,-0.145977,-5.367816,2.805747,-8.173563,-0.937985,-4.0,2.0,2.0,2.0,-1.547126,-0.021814,2.073563,0.003991,-0.900000,-4.666144,0.040565,9.297038,-0.157862,0.011885,-5.0,-1.0,4.0,0.0,-1.0,26.0,-7.0,28.0,-10.0,-0.002805,-0.285,-11,-9,-4,0,11.0,7.0,-1711.0


## 2023 Matchups

In [50]:
seeds = pd.read_csv("data/2023/kaggle_data/MNCAATourneySeeds.csv")


In [51]:
curr = seeds[seeds['Season']==2023]

In [52]:
from itertools import combinations


In [53]:
# get all two-pair combinations of column 'A' as a list
combos = list(combinations(seeds['TeamID'], 2))

unique_pairs = set(frozenset(pair) for pair in combos)


# create a new dataframe from the combinations list
combo_df = pd.DataFrame(unique_pairs, columns=['team1', 'team2'])

In [54]:
combo_df['Season']=2023

In [57]:
combo_df = combo_df[combo_df['team1'] != combo_df['team2']]

In [59]:
features[features['Season']==2023]

,Season,TeamID,Seed,total_games,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,...,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,returning_min_pct,returning_score_pct,preseason_pts
1268,2023,1104,X01,34,0.852941,82.176471,68.500000,13.676471,17.443557,14.0,...,0,1,0.689,6,1,0,0,40.7,41.0,281.0
1269,2023,1112,X02,34,0.823529,82.676471,71.500000,11.176471,14.178423,13.0,...,0,1,0.859,2,1,0,0,45.3,44.6,543.0
1270,2023,1113,Z11a,34,0.647059,70.323529,67.705882,2.617647,13.847945,12.0,...,0,0,0.580,4,0,0,0,48.8,40.4,0.0
1271,2023,1116,Z08,33,0.606061,74.424242,67.393939,7.030303,14.819080,6.0,...,1,0,0.730,6,3,0,0,16.6,13.0,1026.0
1272,2023,1120,Y09,32,0.625000,72.718750,67.062500,5.656250,13.823914,6.0,...,1,0,0.666,12,5,1,0,63.2,56.6,623.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,2023,1433,Z12,34,0.794118,71.441176,62.911765,8.529412,10.968498,12.0,...,0,0,0.611,3,0,0,0,41.4,36.1,0.0
1332,2023,1436,W15,32,0.687500,72.156250,66.718750,5.437500,15.668315,10.0,...,0,0,0.714,5,0,0,0,40.2,33.5,0.0
1333,2023,1438,X04,32,0.781250,67.812500,60.250000,7.562500,11.717875,10.0,...,0,1,0.723,11,4,1,1,87.8,91.4,462.0
1334,2023,1452,X09,33,0.575758,76.303030,71.030303,5.272727,15.531537,6.0,...,0,0,0.690,26,9,2,0,17.1,11.5,0.0


In [60]:
combo_df.merge(features, left_on=['Season', 'team1'], right_on=['Season', 'TeamID'])

,team1,team2,Season,TeamID,Seed,total_games,win_pct,avg_points_for,avg_points_against,avg_game_margin,...,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,returning_min_pct,returning_score_pct,preseason_pts
0,1222,1175.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
1,1222,1191.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
2,1222,1286.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
3,1222,1279.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
4,1222,1151.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11919,1159,1135.0,2023,1159,Y15,33,0.757576,78.151515,69.242424,8.909091,...,0,0,0.539,4,0,0,0,69.2,65.4,0.0
11920,1159,1247.0,2023,1159,Y15,33,0.757576,78.151515,69.242424,8.909091,...,0,0,0.539,4,0,0,0,69.2,65.4,0.0
11921,1159,1199.0,2023,1159,Y15,33,0.757576,78.151515,69.242424,8.909091,...,0,0,0.539,4,0,0,0,69.2,65.4,0.0
11922,1159,1183.0,2023,1159,Y15,33,0.757576,78.151515,69.242424,8.909091,...,0,0,0.539,4,0,0,0,69.2,65.4,0.0


In [61]:
# 2023 matchups

bracket_data = combo_df.merge(features, left_on=['Season', 'team1'], right_on=['Season', 'TeamID'])\
.merge(features, left_on=['Season', 'team2'], right_on=['Season', 'TeamID'], suffixes=['_1', '_2'])

In [62]:
bracket_data.shape

(2278, 89)

In [63]:
difference_cols = ['total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'preseason_pts']
for col in difference_cols:
    bracket_data['diff_'+col] = bracket_data[col+'_1'] - bracket_data[col+'_2']
diff_cols = [col for col in bracket_data.columns if 'diff' in col]
matchup_cols = ['Season', 'TeamID_1', 'TeamID_2', 'Seed_1', 'Seed_2'] + diff_cols
bracket_data = bracket_data[matchup_cols]

In [64]:
bracket_data.shape

(2278, 43)

In [65]:
bracket_data.to_csv('data/2023/bracket_data/bracket_data.csv', index=False)

In [66]:
bracket_data.head(20)

,Season,TeamID_1,TeamID_2,Seed_1,Seed_2,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,...,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_preseason_pts
0,2023,1222,1286,Y01,W14,2,0.193015,3.998162,-10.437500,14.435662,...,-4.0,0.003994,0.027,17,6,2,0,-8.1,-19.2,1404.0
1,2023,1280,1286,Y11a,W14,1,-0.082386,-5.122159,-5.937500,0.815341,...,-4.0,0.000765,0.083,2,0,0,0,-6.7,-13.3,0.0
2,2023,1394,1286,X16b,W14,-3,-0.063578,7.244612,5.993534,1.251078,...,15.0,-0.000962,0.022,0,0,0,0,32.3,30.0,0.0
3,2023,1277,1286,W07,W14,-1,-0.105847,-0.805444,0.449597,-1.255040,...,11.0,0.000938,0.052,23,14,8,1,1.9,-5.0,35.0
4,2023,1243,1286,W03,W14,0,0.000000,4.468750,2.093750,2.375000,...,4.0,0.001527,0.060,-1,0,0,0,-27.0,-29.7,0.0
5,2023,1224,1286,Z16,W14,-1,-0.105847,2.581653,5.675403,-3.093750,...,2.0,-0.001085,-0.233,-1,0,0,0,-5.4,-8.0,0.0
6,2023,1361,1286,X05,W14,0,0.093750,0.312500,-3.125000,3.437500,...,3.0,0.001674,0.101,2,0,0,0,15.0,13.7,394.0
7,2023,1344,1286,W11,W14,0,-0.062500,7.062500,4.375000,2.687500,...,-20.0,0.000969,-0.057,5,1,0,0,-26.7,-28.3,0.0
8,2023,1401,1286,Y07,W14,2,0.016544,2.204044,-0.731618,2.935662,...,10.0,0.001476,-0.038,7,4,0,0,13.4,5.6,112.0
9,2023,1369,1286,X16a,W14,1,-0.203598,6.180871,10.759470,-4.578598,...,-15.0,-0.001104,-0.196,-1,0,0,0,-16.6,-21.2,0.0
